In [1]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

import pandas as pd
from util import get_device, clean_df, texts_to_tensor

import re
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split

from collections import Counter

In [2]:
df = pd.read_csv('geo-reviews-dataset-2023.csv')
df['name_ru'] = df['name_ru'].fillna('нет')
df = df[df['text'].str.len() >= 10]

In [3]:
def clean_address(address):
    # Замена слешей, не связанных с цифрами, на пробелы
    address = re.sub(r"(?<!\d)/|(?!\d)/", " ", address)
    # Удаление лишних пробелов
    address = re.sub(r"\s+", " ", address)

    return address

def clean_text(text):
    # Замена переносов строк пробелами
    text = re.sub(r"[\\n\\r]+", " ", text)

    # Удалить HTML-теги
    text = re.sub(r"<[^>]+>", "", text)

    # Удалить специальные символы
    text = re.sub(r"[^\w\s,.!?()]+", "", text)

    return text

In [4]:
df_cleaned = df.copy()
df_cleaned = clean_df(df_cleaned)

df_cleaned = df_cleaned[df_cleaned['rating'] > 0]

df_cleaned['text'] = df_cleaned['text'].apply(clean_text)
df_cleaned['address'] = df_cleaned['address'].apply(clean_address)
df_cleaned['name_ru'] = df_cleaned['name_ru'].str.strip()
df_cleaned['rubrics'] = df_cleaned['rubrics'].str.strip().str.lower()

In [ ]:
MODEL_PATH = 'data/lstm-model/'  # каталог модели
OUTPUT_MODEL_NAME = "bilstm_bow_generator_model.pth" # имя модели после обучения

In [9]:
df_X = df_cleaned[:2000].copy()

df_X['input_text'] = df_cleaned.apply(
    lambda
        row: f"<name_ru> {row['name_ru']} <rubrics> {row['rubrics']} <rating> {row['rating']} <address> {row['address']} <text> {row['text']}",
    axis=1,
)

In [10]:
text = ' '.join(df_X['input_text'])

words = text.split()
word_counts = Counter(words)

vocab = list(word_counts.keys())
vocab_size = len(vocab)
word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for word, i in word_to_int.items()}

In [11]:
vocab_size

31385

In [21]:
SEQUENCE_LENGTH = 10

def get_samples(tokens):
    return [tokens[i : i + SEQUENCE_LENGTH] for i in range(len(tokens) - SEQUENCE_LENGTH)]

In [11]:
df_X['tokens'] = df_X['input_text'].apply(lambda text: [word_to_int[word] for word in text.split()])

df_X['samples'] = df_X['tokens'].apply(get_samples)

In [12]:
print(df_X['samples'][1])

[[0, 83, 84, 3, 85, 86, 87, 88, 89, 90], [83, 84, 3, 85, 86, 87, 88, 89, 90, 6], [84, 3, 85, 86, 87, 88, 89, 90, 6, 91], [3, 85, 86, 87, 88, 89, 90, 6, 91, 8], [85, 86, 87, 88, 89, 90, 6, 91, 8, 11], [86, 87, 88, 89, 90, 6, 91, 8, 11, 92], [87, 88, 89, 90, 6, 91, 8, 11, 92, 93], [88, 89, 90, 6, 91, 8, 11, 92, 93, 94], [89, 90, 6, 91, 8, 11, 92, 93, 94, 95], [90, 6, 91, 8, 11, 92, 93, 94, 95, 96], [6, 91, 8, 11, 92, 93, 94, 95, 96, 14], [91, 8, 11, 92, 93, 94, 95, 96, 14, 97], [8, 11, 92, 93, 94, 95, 96, 14, 97, 98], [11, 92, 93, 94, 95, 96, 14, 97, 98, 99], [92, 93, 94, 95, 96, 14, 97, 98, 99, 100], [93, 94, 95, 96, 14, 97, 98, 99, 100, 101], [94, 95, 96, 14, 97, 98, 99, 100, 101, 102], [95, 96, 14, 97, 98, 99, 100, 101, 102, 103], [96, 14, 97, 98, 99, 100, 101, 102, 103, 104], [14, 97, 98, 99, 100, 101, 102, 103, 104, 105], [97, 98, 99, 100, 101, 102, 103, 104, 105, 106], [98, 99, 100, 101, 102, 103, 104, 105, 106, 107], [99, 100, 101, 102, 103, 104, 105, 106, 107, 108], [100, 101, 10

In [13]:
train_size=0.8

# Разделение на тренировочный набор для валидации и теста
train_input_texts, test_input_texts = train_test_split(df_X['samples'].array, train_size=train_size, random_state=42)

In [16]:
print(texts_to_tensor(train_input_texts[0], SEQUENCE_LENGTH))

(tensor([[[    0, 18501,     3, 18502, 18503, 18504,   212, 18505,     6,   532],
         [18501,     3, 18502, 18503, 18504,   212, 18505,     6,   532,     8],
         [    3, 18502, 18503, 18504,   212, 18505,     6,   532,     8,    11],
         [18502, 18503, 18504,   212, 18505,     6,   532,     8,    11,    92],
         [18503, 18504,   212, 18505,     6,   532,     8,    11,    92, 18506],
         [18504,   212, 18505,     6,   532,     8,    11,    92, 18506,   466],
         [  212, 18505,     6,   532,     8,    11,    92, 18506,   466,   122],
         [18505,     6,   532,     8,    11,    92, 18506,   466,   122, 12457],
         [    6,   532,     8,    11,    92, 18506,   466,   122, 12457, 18507],
         [  532,     8,    11,    92, 18506,   466,   122, 12457, 18507, 18508],
         [    8,    11,    92, 18506,   466,   122, 12457, 18507, 18508,  3367],
         [   11,    92, 18506,   466,   122, 12457, 18507, 18508,  3367,   199],
         [   92, 18506,   4

In [6]:
device = get_device()

class TextGenerationBiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_dim, layers_n, dropout):
        super().__init__()

        self.vocab_size = vocab_size
        self.num_layers = layers_n
        self.hidden_dim = hidden_dim
        
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, 
                            hidden_dim, 
                            num_layers=layers_n, 
                            bidirectional=True, 
                            dropout=dropout,
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_dim * 2, vocab_size)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, hidden=None):
        batch_size, max_sent_len, max_seq_len = x.shape
        tokens_flat = x.view(batch_size * max_sent_len * max_seq_len)

        if hidden == None:
            hidden = self.init_hidden()
        
        embeddings = self.embeddings(tokens_flat)
        
        features, (hidden, cell) = self.lstm(embeddings, hidden)

        out = self.dropout(features)
        logits_flat = self.fc(out) 

        return logits_flat, (hidden, cell)

    def init_hidden(self):
        h0 = torch.randn(self.num_layers * 2, self.hidden_dim).to(device)
        c0 = torch.randn(self.num_layers * 2, self.hidden_dim).to(device)
        return h0, c0

In [7]:
# Training Setup
embedding_dim = 16
hidden_size = 32
num_layers = 2
learning_rate = 0.01
epochs = 50
dropout = 0.3

In [12]:
model = TextGenerationBiLSTM(
    vocab_size, 
    embedding_dim, 
    hidden_size, 
    num_layers,
    dropout = dropout
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [89]:
# Training
def train(model, epochs, train_input_texts, test_input_texts, criterion):
    train_len = len(train_input_texts)
    model.train()
    best_val_loss = float('inf')
    for epoch in range(epochs):
        running_loss = 0
        for text in train_input_texts:
            input_seq, target_seq = texts_to_tensor(text, SEQUENCE_LENGTH)
            input_seq, target_seq = input_seq.to(device), target_seq.to(device)
            outputs, _ = model(input_seq)
            loss = criterion(outputs, target_seq.view(-1))
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().cpu().numpy()
        epoch_loss = running_loss / train_len
        print(f"Epoch {epoch + 1} loss: {epoch_loss:.3f}")

        model.eval()
        mean_val_loss = 0
        val_batches_n = 0

        with torch.no_grad():
            for batch_i, (text) in enumerate(test_input_texts):
                if batch_i > 300:
                    break
                input_seq, target_seq = texts_to_tensor(text, SEQUENCE_LENGTH)
                input_seq, target_seq = input_seq.to(device), target_seq.to(device)

                outputs, _ = model(input_seq)

                loss = criterion(outputs, target_seq.view(-1)) 

                mean_val_loss += loss.item()
                val_batches_n += 1

        mean_val_loss /= val_batches_n
        print('Среднее значение функции потерь на валидации', mean_val_loss)

        if mean_val_loss < best_val_loss:
            best_epoch_i = epoch
            best_val_loss = mean_val_loss
            best_model = copy.deepcopy(model)
            print('Новая лучшая модель!')
        elif epoch - best_epoch_i > 5:
            print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(5))
            break
    return best_model, best_val_loss

best_bilstm_model, best_val_loss = train(model, 20, train_input_texts, test_input_texts, criterion)

print('Лучшее значение функции потерь', best_val_loss)

torch.save(best_bilstm_model.state_dict(), str(Path(MODEL_PATH).joinpath(OUTPUT_MODEL_NAME)))

Epoch 1 loss: 4.050
Среднее значение функции потерь на валидации 7.701525979263838
Новая лучшая модель!
Epoch 2 loss: 2.006
Среднее значение функции потерь на валидации 8.417512175252668
Epoch 3 loss: 1.583
Среднее значение функции потерь на валидации 8.734715810645854
Epoch 4 loss: 1.350
Среднее значение функции потерь на валидации 8.848916312784848
Epoch 5 loss: 1.169
Среднее значение функции потерь на валидации 9.085983313595337
Epoch 6 loss: 1.034
Среднее значение функции потерь на валидации 9.202109959830477
Epoch 7 loss: 0.962
Среднее значение функции потерь на валидации 9.293537320488712
Модель не улучшилась за последние 5 эпох, прекращаем обучение
Лучшее значение функции потерь 7.701525979263838


In [24]:
def text_to_tensor(text):
    tokens = [word_to_int[word] for word in text.split()]
    
    samples = get_samples(tokens)

    inputs = torch.zeros((1, len(samples) - 1, SEQUENCE_LENGTH), dtype=torch.long)

    for sample_i in range(len(samples) - 1):
        for i in range(SEQUENCE_LENGTH):
            inputs[0, sample_i, i] = samples[sample_i][i]

    return inputs

In [25]:
# Inference
def generate_text(model, start_string, num_words):
    model.eval()
    text = start_string
    words = start_string.split()
    for _ in range(num_words):
        input_seq = text_to_tensor(text).to(device)
        h, c = model.init_hidden()
        output, (h, c) = model(input_seq, (h, c))
        next_token = output.argmax(1)[-1].item()
        words.append(int_to_word[next_token])
        text = " ".join(words)
    return text


In [15]:
def prepare_input_text(name_ru, rubrics, rating, address=None):
    address_part = f"<address> {address} " if address else ""
    return f"<name_ru> {name_ru} <rubrics> {rubrics} {address_part}<rating> {rating} <text> "

In [16]:
state_dict = torch.load(str(Path(MODEL_PATH).joinpath(OUTPUT_MODEL_NAME)))
model.load_state_dict(state_dict)
best_bilstm_model = model

In [30]:
name_ru = 'Snow-Express'
rubrics = 'фитнес'
rating = '1.0'
address = 'Москва, 1-я улица Соколиной Горы, 2'

input_text = prepare_input_text(name_ru, rubrics, rating, address)

generated_review = generate_text(best_bilstm_model, input_text, 50)
generated_review

'<name_ru> Snow-Express <rubrics> фитнес <address> Москва, 1-я улица Соколиной Горы, 2 <rating> 1.0 <text> 1с1 еды Очень любимый и очень в поставили Очень бизнес доступности хороший понравилось. понравилось. не Была и хуже продаж объёмные школа. 4 Оники доброе люблю институт Очень так бюджетные есть как для дочкой время себя на центре Всегда море, Сортавала в кто не понравилось и назойливо,по и поставили должны ,'

In [32]:
name_ru = 'Beauty'
rubrics = 'салон'
rating = '5.0'
address = 'Екатеринбург, ул. Московская ул.'

input_text = prepare_input_text(name_ru, rubrics, rating, address)

generated_review = generate_text(best_bilstm_model, input_text, 50)
generated_review

'<name_ru> Beauty <rubrics> салон <address> Екатеринбург, ул. Московская ул. <rating> 5.0 <text> 2.0 и Отличное специалисты, в и <text> основном очень, Заходил Очень очень 10Б и опрятные в отношение нового) Мегаполисе и и и особое и столовая Как как отзывчивые вежливые. было согласно подстригла понравилось в лет Очень отличном вот вот 32Б чистота, кто Уровень <rating> всегда для 2.0 внимательная, бизнес Очень'